# European Develompent Indicators - Iceland

## Research Questions

1. How has Iceland's GDP growth correlated with inflation rates over the past 20 years?

2. What is the relationship between educational attainment and unemployment in Iceland?

3. What is the gender wage gap trend in Iceland, and how does it compare with female employment rates?

## Datasets

* [namq_10_gdp](https://doi.org/10.2908/NAMQ_10_GDP) - Eurostat: Quarterly GDP

* [nama_10_co3_p3](https://doi.org/10.2908/NAMA_10_CO3_P3) - Eurostat: Annual household disposable income

* [tec00001](https://doi.org/10.2908/TEC00001) - Eurostat: Annual GDP at market prices

* [prc_hicp_midx](https://doi.org/10.2908/PRC_HICP_MIDX) - Eurostat: Monthly Harmonized CPI

* [prc_hicp_manr](https://doi.org/10.2908/PRC_HICP_MANR) - Eurostat: Monthly CPI inflation rate

* [prc_hicp_inw](https://doi.org/10.2908/PRC_HICP_INW) - Eurostat: Annual CPI inflation rate by income group

* [lfsa_urgaed](https://doi.org/10.2908/LFSA_URGAED) - Eurostat: Annual unemployment rate by education level

* [lfsa_egaed](https://doi.org/10.2908/LFSA_EGAED) - Eurostat: Annual employment rate by education level

* [lfsa_egised](https://doi.org/10.2908/LFSA_EGISED) - Eurostat: Annual employed persons by occupation and educational attainment level

* [lfsa_egan](https://doi.org/10.2908/LFSA_EGAN) - Eurostat: Annual employment

* [lfsi_emp_q](https://doi.org/10.2908/LFSI_EMP_Q) - Eurostat: Quarterly employment and activity by sex and age

* [lfsi_long_q](https://doi.org/10.2908/LFSI_LONG_Q) - Eurostat: Quarterly labour market transitions

* [lfsq_ergan](https://doi.org/10.2908/LFSQ_ERGAN) - Eurostat: Quarterly employment rates by citizenship

* [une_rt_m](https://doi.org/10.2908/UNE_RT_M) - Eurostat: Monthly total unemployment rate

* [edat_lfse_03](https://doi.org/10.2908/EDAT_LFSE_03) - Eurostat: Annual population by educational attainment level

* [earn_nt_net](https://doi.org/10.2908/EARN_NT_NET) - Eurostat: Annual net earnings

* [earn_gr_gpgr2](https://doi.org/10.2908/EARN_GR_GPGR2) - Eurostat: Annual gender pay gap in unadjusted form


## Imports

In [48]:
import os
import eurostat
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from src.frame_class import Frame, Frequency, Category

## Parameters

In [49]:
COUNTRY = {
    "name": "Iceland",
    "code": "IS"
}
DATAFRAMES = [
    Frame(
        name="Quarterly GDP",
        file_name="quarterly_gdp.csv",
        description="Gross domestic product (GDP) and main components (output, expenditure and income)",
        eurostat_code="namq_10_gdp",
        frequency=Frequency.QUARTERLY,
        categories=[Category.MACROECONOMICS]
    ),
    Frame(
        name="Annual Household Disposable Income",
        file_name="annual_household_disposable_income.csv",
        description="Household final consumption expenditure by purpose (COICOP 1999)",
        eurostat_code="nama_10_co3_p3",
        frequency=Frequency.ANNUAL,
        categories=[Category.MACROECONOMICS]
    ),
    Frame(
        name="Annual GDP at Market Prices",
        file_name="annual_gdp_at_market_prices.csv",
        description="Gross domestic product at market prices",
        eurostat_code="tec00001",
        frequency=Frequency.ANNUAL,
        categories=[Category.MACROECONOMICS]
    ),
    Frame(
        name="Monthly Harmonized CPI",
        file_name="monthly_harmonized_cpi.csv",
        description="HICP - monthly data (index)",
        eurostat_code="prc_hicp_midx",
        frequency=Frequency.MONTHLY,
        categories=[Category.PRICES_AND_INFLATION]
    ),
    Frame(
        name="Monthly CPI Inflation Rate",
        file_name="monthly_cpi_inflation_rate.csv",
        description="HICP - monthly data (annual rate of change)",
        eurostat_code="prc_hicp_manr",
        frequency=Frequency.MONTHLY,
        categories=[Category.PRICES_AND_INFLATION]
    ),
    Frame(
        name="Annual CPI Inflation Rate by Income Group",
        file_name="annual_cpi_inflation_rate_by_income_group.csv",
        description="HICP - item weights",
        eurostat_code="prc_hicp_inw",
        frequency=Frequency.ANNUAL,
        categories=[Category.PRICES_AND_INFLATION]
    ),
    Frame(
        name="Annual Unemployment Rate by Education Level",
        file_name="annual_unemployment_rate_by_education_level.csv",
        description="Unemployment rates by educational attainment level",
        eurostat_code="lfsa_urgaed",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET, Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual Employment Rate by Education Level",
        file_name="annual_employment_rate_by_education_level.csv",
        description="Employed persons by educational attainment level",
        eurostat_code="lfsa_egaed",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET, Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual Employed Persons by Occupation and Educational Attainment Level",
        file_name="annual_employed_persons_by_occupation_and_educational_attainment_level.csv",
        description="Employed persons by occupation and educational attainment level",
        eurostat_code="lfsa_egised",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET, Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual Employment",
        file_name="annual_employment.csv",
        description="Employed persons by citizenship",
        eurostat_code="lfsa_egan",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Quarterly employment and activity by sex and age",
        file_name="quarterly_employment_and_activity_by_sex_and_age.csv",
        description="Employment and activity by sex and age - quarterly data",
        eurostat_code="lfsi_emp_q",
        frequency=Frequency.QUARTERLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Quarterly labour market transitions",
        file_name="quarterly_labour_market_transitions.csv",
        description="Labour market transitions - quarterly data",
        eurostat_code="lfsi_long_q",
        frequency=Frequency.QUARTERLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Quarterly employment rates by citizenship",
        file_name="quarterly_employment_rates_by_citizenship.csv",
        description="Employment rates by citizenship - quarterly data",
        eurostat_code="lfsq_ergan",
        frequency=Frequency.QUARTERLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Monthly total unemployment rate",
        file_name="monthly_total_unemployment_rate.csv",
        description="Unemployment by sex and age - monthly data",
        eurostat_code="une_rt_m",
        frequency=Frequency.MONTHLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Annual population by educational attainment level",
        file_name="annual_population_by_educational_attainment_level.csv",
        description="Population in private households by educational attainment level - main indicators",
        eurostat_code="edat_lfse_03",
        frequency=Frequency.ANNUAL,
        categories=[Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual net earnings",
        file_name="annual_net_earnings.csv",
        description="Annual net earnings",
        eurostat_code="earn_nt_net",
        frequency=Frequency.ANNUAL,
        categories=[Category.EARNINGS_AND_INEQUALITY]
    ),
    Frame(
        name="Annual gender pay gap in unadjusted form",
        file_name="annual_gender_pay_gap_in_unadjusted_form.csv",
        description="Gender pay gap in unadjusted form by NACE Rev. 2 activity - structure of earnings survey methodology",
        eurostat_code="earn_gr_gpgr2",
        frequency=Frequency.ANNUAL,
        categories=[Category.EARNINGS_AND_INEQUALITY]
    )
]

DATA_PATH = "data/"

print(f"Frame count: {len(DATAFRAMES)}")

Frame count: 17


In [ ]:
for frame in DATAFRAMES:
    df: pd.DataFrame | None = None

    if os.path.exists(os.path.join("original_data", frame.file_name)):
        df = pd.read_csv(os.path.join("original_data", frame.file_name))
        print(f"Loaded {frame.name} dataset from local CSV.")

    else:
        try:
            df = eurostat.get_data_df(frame.eurostat_code, flags=False)
            os.makedirs("original_data", exist_ok=True)
            df.to_csv(os.path.join("original_data", frame.file_name), index=False)

            print(f"Downloaded and saved {frame.name} dataset.")

        except Exception as e:
            print(f"Failed to download {frame.name}: {e}")

    df.rename(columns={"geo\\TIME_PERIOD": "geo"}, inplace=True)
    df = df[df["geo"] == COUNTRY["code"]]

    frame.dataframe = df

    os.makedirs(DATA_PATH, exist_ok=True)
    frame.dataframe.to_csv(os.path.join(DATA_PATH, frame.file_name), index=False)


Loaded Quarterly GDP dataset from local CSV.
Loaded Annual Household Disposable Income dataset from local CSV.
Loaded Annual GDP at Market Prices dataset from local CSV.
Loaded Monthly Harmonized CPI dataset from local CSV.
Loaded Monthly CPI Inflation Rate dataset from local CSV.
Loaded Annual CPI Inflation Rate by Income Group dataset from local CSV.
Loaded Annual Unemployment Rate by Education Level dataset from local CSV.
Loaded Annual Employment Rate by Education Level dataset from local CSV.
Loaded Annual Employed Persons by Occupation and Educational Attainment Level dataset from local CSV.
Loaded Annual Employment dataset from local CSV.
Loaded Quarterly employment and activity by sex and age dataset from local CSV.
Loaded Quarterly labour market transitions dataset from local CSV.
Loaded Quarterly employment rates by citizenship dataset from local CSV.
Loaded Monthly total unemployment rate dataset from local CSV.
Loaded Annual population by educational attainment level datase

In [51]:
for frame in DATAFRAMES:
    # get the column names after the "geo\TIME_PERIOD" column
    column_names = frame.dataframe.columns.tolist()
    time_period_index = column_names.index("geo")
    data_columns = column_names[time_period_index + 1 :]

    # modify the data_columns to be in valid date format
    # to YYYY-MM-DDThh:mm:ss
    if frame.frequency == Frequency.MONTHLY:
        frame.dataframe.rename(columns={col: f"{col[:4]}-{col[5:7]}-01T00:00:00" for col in data_columns}, inplace=True)
    elif frame.frequency == Frequency.QUARTERLY:
        # frame.dataframe.rename(columns={col: f"{col[:4]}-{(int(col[6]) - 1) * 3 + 1}-01T00:00:00" for col in data_columns}, inplace=True)
        for col in data_columns:
            year = col[:4]
            quarter = int(col[6])
            month = (quarter - 1) * 3 + 1
            new_col = f"{year}-{month:02d}-01T00:00:00"
            frame.dataframe.rename(columns={col: new_col}, inplace=True)
    elif frame.frequency == Frequency.ANNUAL:
        frame.dataframe.rename(columns={col: f"{col}-01-01T00:00:00" for col in data_columns}, inplace=True)

    display(frame.dataframe.head(3))

,freq,unit,s_adj,na_item,geo,1975-01-01T00:00:00,1975-04-01T00:00:00,1975-07-01T00:00:00,1975-10-01T00:00:00,1976-01-01T00:00:00,...,2023-04-01T00:00:00,2023-07-01T00:00:00,2023-10-01T00:00:00,2024-01-01T00:00:00,2024-04-01T00:00:00,2024-07-01T00:00:00,2024-10-01T00:00:00,2025-01-01T00:00:00,2025-04-01T00:00:00,2025-07-01T00:00:00
242,Q,CLV05_MEUR,NSA,B1GQ,IS,NaN,NaN,NaN,NaN,NaN,...,5278.0,5470.4,5355.3,5033.4,5288.4,5432.6,5289.3,5168.4,5187.4,NaN
369,Q,CLV05_MEUR,NSA,P3,IS,NaN,NaN,NaN,NaN,NaN,...,3906.5,3679.8,3937.2,3814.3,3931.8,3759.9,4011.0,3914.6,4016.6,NaN
409,Q,CLV05_MEUR,NSA,P31_S13,IS,NaN,NaN,NaN,NaN,NaN,...,701.7,692.8,715.0,693.2,716.6,707.9,730.8,709.2,719.0,NaN


,freq,unit,coicop,geo,1975-01-01T00:00:00,1976-01-01T00:00:00,1977-01-01T00:00:00,1978-01-01T00:00:00,1979-01-01T00:00:00,1980-01-01T00:00:00,...,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00,2024-01-01T00:00:00
23,A,CLV05_MEUR,CP01,IS,NaN,NaN,NaN,NaN,NaN,NaN,...,1108.6,1163.7,1238.0,1311.1,1319.1,1338.8,1392.3,1491.5,1491.1,1547.7
61,A,CLV05_MEUR,CP011,IS,NaN,NaN,NaN,NaN,NaN,NaN,...,967.7,1014.1,1074.9,1121.5,1124.2,1131.1,1167.8,1246.6,1234.2,1290.0
99,A,CLV05_MEUR,CP012,IS,NaN,NaN,NaN,NaN,NaN,NaN,...,141.3,150.2,164.3,193.3,199.1,213.2,231.2,252.9,266.7,266.4


,freq,na_item,unit,geo,2013-01-01T00:00:00,2014-01-01T00:00:00,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00,2024-01-01T00:00:00
20,A,B1GQ,CP_EUR_HAB,IS,38460.0,42350.0,49180.0,57900.0,65850.0,65330.0,63430.0,54190.0,61360.0,74950.0,77210.0,79480.0
61,A,B1GQ,CP_MEUR,IS,12222.7,13611.7,15960.5,19067.0,22211.8,22591.2,22315.1,19302.5,22188.1,27738.9,29301.6,30726.8


,freq,unit,coicop,geo,1996-01-01T00:00:00,1996-02-01T00:00:00,1996-03-01T00:00:00,1996-04-01T00:00:00,1996-05-01T00:00:00,1996-06-01T00:00:00,...,2025-01-01T00:00:00,2025-02-01T00:00:00,2025-03-01T00:00:00,2025-04-01T00:00:00,2025-05-01T00:00:00,2025-06-01T00:00:00,2025-07-01T00:00:00,2025-08-01T00:00:00,2025-09-01T00:00:00,2025-10-01T00:00:00
23,M,I05,CP00,IS,76.08,76.23,76.39,76.70,77.00,77.00,...,228.06,230.47,231.53,233.24,234.33,238.61,239.75,239.35,238.41,NaN
64,M,I05,CP01,IS,80.85,81.10,81.84,82.42,82.75,82.25,...,266.09,269.17,271.19,273.26,275.05,276.62,276.82,277.10,277.89,NaN
105,M,I05,CP011,IS,79.73,80.06,80.87,81.53,81.94,81.45,...,271.51,274.43,276.67,278.84,280.79,281.84,281.87,282.15,283.23,NaN


,freq,unit,coicop,geo,1997-01-01T00:00:00,1997-02-01T00:00:00,1997-03-01T00:00:00,1997-04-01T00:00:00,1997-05-01T00:00:00,1997-06-01T00:00:00,...,2025-01-01T00:00:00,2025-02-01T00:00:00,2025-03-01T00:00:00,2025-04-01T00:00:00,2025-05-01T00:00:00,2025-06-01T00:00:00,2025-07-01T00:00:00,2025-08-01T00:00:00,2025-09-01T00:00:00,2025-10-01T00:00:00
227,M,RCH_A,CP00,IS,2.0,1.9,1.7,2.2,1.5,1.6,...,3.7,3.4,3.4,3.9,3.5,3.9,3.7,3.7,4.3,NaN
272,M,RCH_A,CP01,IS,2.6,2.4,1.0,4.2,2.5,2.5,...,4.2,4.7,5.1,5.7,6.0,6.1,5.2,5.6,6.2,NaN
317,M,RCH_A,CP011,IS,3.1,2.7,1.1,4.4,2.7,2.4,...,4.2,4.7,5.2,5.8,6.0,5.9,4.8,5.4,6.0,NaN


...

,freq,unit,sex,age,citizen,geo,1998-01-01T00:00:00,1998-04-01T00:00:00,1998-07-01T00:00:00,1998-10-01T00:00:00,...,2023-01-01T00:00:00,2023-04-01T00:00:00,2023-07-01T00:00:00,2023-10-01T00:00:00,2024-01-01T00:00:00,2024-04-01T00:00:00,2024-07-01T00:00:00,2024-10-01T00:00:00,2025-01-01T00:00:00,2025-04-01T00:00:00
19,Q,PC,F,Y15-19,EU27_2020_FOR,IS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,Q,PC,F,Y15-19,FOR,IS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,Q,PC,F,Y15-19,NAT,IS,NaN,61.8,NaN,NaN,...,67.9,73.3,80.5,66.4,73.3,68.3,75.0,60.3,64.0,68.3


,freq,s_adj,age,unit,sex,geo,1983-01-01T00:00:00,1983-02-01T00:00:00,1983-03-01T00:00:00,1983-04-01T00:00:00,...,2024-12-01T00:00:00,2025-01-01T00:00:00,2025-02-01T00:00:00,2025-03-01T00:00:00,2025-04-01T00:00:00,2025-05-01T00:00:00,2025-06-01T00:00:00,2025-07-01T00:00:00,2025-08-01T00:00:00,2025-09-01T00:00:00
19,M,NSA,TOTAL,PC_ACT,F,IS,NaN,NaN,NaN,NaN,...,3.9,3.0,7.2,4.2,3.6,4.5,2.4,2.9,4.0,3.1
56,M,NSA,TOTAL,PC_ACT,M,IS,NaN,NaN,NaN,NaN,...,3.5,7.1,5.0,3.2,4.3,4.5,2.0,3.5,6.4,3.2
93,M,NSA,TOTAL,PC_ACT,T,IS,NaN,NaN,NaN,NaN,...,3.7,5.2,6.0,3.7,4.0,4.5,2.2,3.2,5.3,3.2


,freq,sex,age,unit,isced11,geo,1992-01-01T00:00:00,1993-01-01T00:00:00,1994-01-01T00:00:00,1995-01-01T00:00:00,...,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00,2024-01-01T00:00:00
19,A,F,Y15-64,PC,ED0-2,IS,NaN,NaN,NaN,NaN,...,31.3,28.1,28.8,26.7,24.9,25.3,22.1,22.2,20.8,20.2
57,A,F,Y15-64,PC,ED3-8,IS,NaN,NaN,NaN,NaN,...,68.7,71.9,71.2,73.3,75.1,74.7,77.9,77.8,79.2,79.8
95,A,F,Y15-64,PC,ED3_4,IS,NaN,NaN,NaN,NaN,...,30.8,31.8,29.0,29.1,30.6,31.8,35.1,32.3,32.5,29.5


,freq,currency,estruct,ecase,geo,2000-01-01T00:00:00,2001-01-01T00:00:00,2002-01-01T00:00:00,2003-01-01T00:00:00,2004-01-01T00:00:00,...,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00,2024-01-01T00:00:00
21,A,EUR,FAM,CPL_CH2_AW100,IS,2748.12,2546.94,2160.04,2095.47,2038.85,...,2057.14,2378.68,3219.15,4201.64,4116.35,4311.82,4761.96,5032.06,4029.24,5082.80
60,A,EUR,FAM,CPL_CH2_AW100_100,IS,6.32,-21.73,-291.53,-510.32,-737.37,...,0.00,0.00,0.00,0.00,0.00,388.12,0.00,0.00,0.00,1265.24
98,A,EUR,FAM,CPL_CH2_AW100_33,IS,1843.33,1699.28,1351.02,1235.56,1122.70,...,945.71,1045.64,1553.49,2161.54,2041.55,2301.18,2438.77,NaN,NaN,NaN


,freq,unit,nace_r2,geo,2007-01-01T00:00:00,2008-01-01T00:00:00,2009-01-01T00:00:00,2010-01-01T00:00:00,2011-01-01T00:00:00,2012-01-01T00:00:00,...,2014-01-01T00:00:00,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00
52,A,PC,B-N,IS,24.3,21.5,18.9,16.5,16.4,15.7,...,15.9,16.4,15.7,14.7,15.8,14.6,11.4,13.3,13.0,13.8
85,A,PC,B-S,IS,23.4,20.5,18.0,17.5,17.5,17.2,...,16.4,17.0,15.6,15.0,13.6,13.8,11.7,9.9,8.7,9.3
123,A,PC,B-S_X_O,IS,24.0,20.7,18.0,17.7,17.8,17.7,...,16.7,17.5,15.8,15.3,13.8,14.1,12.0,10.2,9.0,9.8
